# Warm up
Let's start with some warm up exercises to get you familiar with the database and do some SQL querying.
Make sure to have downloaded the NBA dataset form Kaggle as described in the README.

In [ ]:
import sqlite3 as sql
import pandas as pd
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


## TODO: Use pathlib to get the path to the data directory. Path(__file__) won't work in a Jupyter notebook.
DATA_PATH = Path.cwd() / 'data/nba.sqlite'

## Create a SQLite connection and run some SQL queries

In [ ]:
con = sql.connect(DATA_PATH)  # connect to the database
query = 'SELECT * FROM game LIMIT 10'  # write a query that selects the first 10 rows from the game table
top_10_game = pd.read_sql(query, con)  # read the query into a pandas dataframe
top_10_game.head() # print the dataframe

In [ ]:
# let's look at the schema of the database
# TODO: sqlite_master is the name of the table you want to inspeact: Replace TABLE_NAME with the name of the table you want to inspect
query = """
    SELECT *
    FROM sqlite_master
"""

pd.read_sql(query, con).sort_values('name')

In [ ]:
# let's see all the tables we have in the database
# TODO: Filter for tables only
query = """
    SELECT name
    FROM sqlite_master
    WHERE type = 'table'
"""
pd.read_sql(query, con)

In [ ]:
# let's have a look at the columns in the game table
query = """
    PRAGMA table_info(draft_combine_stats)
"""
pd.read_sql(query, con)

In [ ]:
# let's find the earliest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date
    LIMIT 1
"""

print(pd.read_sql(query, con))

# let's find the latest recorded game
query = """
    SELECT game_date
    FROM game
    ORDER BY game_date DESC
    LIMIT 1
"""

print(pd.read_sql(query, con))


In [ ]:
# TODO: Find four interesting facts about the data and write a query to find the answer

In [ ]:
#sight 1
query = """
    SELECT height_w_shoes AS height, max_vertical_leap AS vertical_leap
    FROM draft_combine_stats
    WHERE height_w_shoes IS NOT NULL AND max_vertical_leap IS NOT NULL
"""
df1 = pd.read_sql(query, con)
df1


To see if taller players generally have a lower or higher vertical leap, we can analyze the height_w_shoes and max_vertical_leap columns. This insight could reveal if taller players tend to have lower verticals or if there's no significant correlation.

In [ ]:
# Assuming df is the DataFrame resulting from the query
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df1, x='height', y='vertical_leap')
plt.title('Correlation between Height and Vertical Leap')
plt.xlabel('Height')
plt.ylabel('Max Vertical Leap')
plt.grid(True)
plt.show()

In [ ]:
#sight 2
query = """
    SELECT position, player_name, three_quarter_sprint
    FROM draft_combine_stats
    WHERE three_quarter_sprint IS NOT NULL
    ORDER BY three_quarter_sprint ASC
    LIMIT 10
"""
df2 = pd.read_sql(query, con)
df2


Examining the three_quarter_sprint column can help identify the fastest sprint times by player position. This insight would highlight which positions tend to have the fastest sprint speeds and which individual players stand out.

In [ ]:
#sight 3
query = """

    SELECT 
        AVG(CAST(spot_nba_corner_left AS FLOAT)) AS nba_corner_left_accuracy,
        AVG(CAST(spot_nba_top_key AS FLOAT)) AS nba_top_key_accuracy,
        AVG(CAST(spot_college_top_key AS FLOAT)) AS college_top_key_accuracy,
        AVG(CAST(off_drib_college_top_key AS FLOAT)) AS off_drib_college_accuracy
    FROM draft_combine_stats
    WHERE 
        spot_nba_corner_left IS NOT NULL 
        AND spot_nba_top_key IS NOT NULL
        AND spot_college_top_key IS NOT NULL
        AND off_drib_college_top_key IS NOT NULL
"""
df3 = pd.read_sql(query, con)
df3


In [ ]:
# Reshape the DataFrame for easier plotting (if necessary)
accuracy_df = pd.DataFrame({
    'Shot Type': ['NBA Corner Left', 'NBA Top Key', 'College Top Key', 'Off-Dribble College'],
    'Accuracy': [df3['nba_corner_left_accuracy'].iloc[0],
                 df3['nba_top_key_accuracy'].iloc[0],
                 df3['college_top_key_accuracy'].iloc[0],
                 df3['off_drib_college_accuracy'].iloc[0]]
})

plt.figure(figsize=(10, 6))
sns.barplot(data=accuracy_df, x='Shot Type', y='Accuracy', palette='viridis')
plt.title('Shooting Accuracy by Shot Type')
plt.xlabel('Shot Type')
plt.ylabel('Average Shooting Accuracy (%)')
plt.ylim(0, 100)
plt.show()
